### Instalar:
pip install elasticsearch

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, BulkIndexError

### Configuración elastic:
1. Después del login ir a https://cloud.elastic.co/home y entrar en "My deployment".
2. Haga click en las trtes rallitas para abrir "Manager this deployment" y luego vaya a "API Keys" para generar un nuevo API Key.
3. Para identifiacar el "Cloud ID" , vaya a "My deployment" y haga click en "Manage".
4. Para "Elasticsearch endpoint" ve a "My deployment", luego a "Elasticsearch" y "Copy endpoint". Tendras algo como esto en el portapapeles.
    https://52ce5dc4d987461e90effb1ad06a6068.us-central1.gcp.cloud.es.io


In [1]:
from elasticsearch import Elasticsearch


es = Elasticsearch(
    "https://devfest-lima-2024.es.us-central1.gcp.cloud.es.io",  # Elasticsearch endpoint
    api_key="UkkyWGRwTUJud2NwNG9iTlMxUHo6TjhtLUR3R2VUcU90M0l1R1ZlalR3dw==",
)
# Successful response!
es.info().body

{'name': 'instance-0000000000',
 'cluster_name': '47ca47e25f024c7784b2d65dfae50378',
 'cluster_uuid': 'lzKsF-dPTmyqLALkDYr5dQ',
 'version': {'number': '8.15.5',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'b10896bcfe167cce44a84ba2771d101fb596d40d',
  'build_date': '2024-11-21T22:06:13.985834967Z',
  'build_snapshot': False,
  'lucene_version': '9.11.1',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [2]:
import pickle

# Replace 'path_to_your_file.pkl' with the actual path to your PKL file
file_path = 'data/chapter_1_256.pkl'

# Open the file in binary mode and load the data
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Now 'data' contains the deserialized Python object
print(len(data))

160


In [3]:
all_data =[]

for idx, texto in enumerate(data):
    tem =   {'id': idx+1,  
            'title': '100 años de soledad',  
            'description': texto
            }
    all_data.append(tem)

In [4]:
len(all_data)

160

In [7]:
all_data[11]

{'id': 12,
 'title': '100 años de soledad',
 'description': 'Cuando José Arcadio Buendía y los cuatro hombres de su expedición lograron desarticular la armadura, encontraron dentro un esqueleto calcificado que llevaba colgado en el cuello un relicario de cobre con un rizo de mujer. En marzo volvieron los gitanos.'}

In [8]:
index_settings = {
        "settings": {
            "analysis": {"analyzer": {"default": {"type": "spanish"}}},
            "similarity": {"default": {"type": "BM25"}},
            "index.queries.cache.enabled": False  # Disable query cache
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword", "index": False},
                "title": {"type": "text", "analyzer": "spanish"},
                "description": {"type": "text", "analyzer": "spanish"},
        }},
    }

In [9]:
from elasticsearch.helpers import bulk, BulkIndexError
# Create an index
py_indexname = '100_years_256'
es.indices.create(index=py_indexname, body=index_settings)

# Check if an index exists
if es.indices.exists(index=py_indexname): # he includo index_settings = {...}
  print("Index exists")
else:
  print("Index doesn't exist.")

actions = [
    {"_index": py_indexname,
     "_id": doc["id"],
     "_source": {
          "library": doc["title"],
          "description": doc["description"]}
     }
     for doc in all_data 
]

try:
    bulk(es, actions)
    print("Data successfully indexed in the destination index.")
except BulkIndexError as e: 
    print("Failed to index documents:") 
    for err in e.errors:
        print(err)

Index exists
Data successfully indexed in the destination index.


In [ ]:
q = "doblones"
k = 3
search_body = {
    "query": {
        "multi_match": {
            "query": q,
            "fields": ["description"],
        }
    },
    "size": k,
}
#s.count(index=py_indexname, query=query)
response = es.search(index=py_indexname, body=search_body) # Extract the results
results = response["hits"]["hits"]
# Print the documents
for result in results: 
    print(f"- {result['_score']:3f} -> {result['_source']['description']}")

- 4.000544 -> Úrsula cedió, como ocurría siempre, ante la inquebrantable obstinación de su marido. Entonces José Arcadio Buendía echó treinta doblones en una cazuela, y los fundió con raspadura de cobre, oropimente, azufre y plomo.
- 3.767782 -> Por último, cansado de esperar, se lamentó ante Melquíades del fracaso de su iniciativa, y el gitano dio entonces una prueba convincente de honradez: le devolvió los doblones a cambio de la lupa, y le dejó además unos mapas portugueses y varios instrumentos de navegación.
